In [108]:
import numpy as np
nx = None
import qpms
import spherical_functions as sf, quaternion
qpms.__version__

'0.2.14'

In [470]:
lMax=2
tc = qpms.trans_calculator(lMax)
my, ny = qpms.get_mn_y(lMax)
nelem = len(my)
J=2

In [471]:
qpms.yflip_tyty(lMax).reshape((2*nelem,2*nelem))

array([[ 0,  0, -1,  0,  0,  0,  0,  0,  0,  0,  0,  0,  0,  0,  0,  0],
       [ 0,  1,  0,  0,  0,  0,  0,  0,  0,  0,  0,  0,  0,  0,  0,  0],
       [-1,  0,  0,  0,  0,  0,  0,  0,  0,  0,  0,  0,  0,  0,  0,  0],
       [ 0,  0,  0,  0,  0,  0,  0,  1,  0,  0,  0,  0,  0,  0,  0,  0],
       [ 0,  0,  0,  0,  0,  0, -1,  0,  0,  0,  0,  0,  0,  0,  0,  0],
       [ 0,  0,  0,  0,  0,  1,  0,  0,  0,  0,  0,  0,  0,  0,  0,  0],
       [ 0,  0,  0,  0, -1,  0,  0,  0,  0,  0,  0,  0,  0,  0,  0,  0],
       [ 0,  0,  0,  1,  0,  0,  0,  0,  0,  0,  0,  0,  0,  0,  0,  0],
       [ 0,  0,  0,  0,  0,  0,  0,  0,  0,  0,  1,  0,  0,  0,  0,  0],
       [ 0,  0,  0,  0,  0,  0,  0,  0,  0, -1,  0,  0,  0,  0,  0,  0],
       [ 0,  0,  0,  0,  0,  0,  0,  0,  1,  0,  0,  0,  0,  0,  0,  0],
       [ 0,  0,  0,  0,  0,  0,  0,  0,  0,  0,  0,  0,  0,  0,  0, -1],
       [ 0,  0,  0,  0,  0,  0,  0,  0,  0,  0,  0,  0,  0,  0,  1,  0],
       [ 0,  0,  0,  0,  0,  0,  0,  0,  0,  0,  0,

In [553]:
#d1_cart = np.array([1.02,0,0])
d1_cart = np.random.normal(scale=10, size=(3,)) # translation vector in basis 1
rot21_cart = np.array([np.pi*2**(.5),np.pi*2**(.5),0])
#rot21_cart= np.random.normal(size=(3,)) # random rotation vector
rot21_quat = quaternion.from_rotation_vector(rot21_cart)
rot21_matrix = quaternion.as_rotation_matrix(rot21_quat) # rotation matrix from basis 1 to basis 2
d2_cart = np.dot(rot21_matrix, d1_cart) # translation vector in basis 2
D21_yy = qpms.WignerD_yy_fromvector(lMax,rot21_cart)
D12_yy = qpms.WignerD_yy_fromvector(lMax,-rot21_cart)

d1_sph = qpms.cart2sph(d1_cart)
r1, th1, fi1 = d1_sph
d2_sph = qpms.cart2sph(d2_cart)
r2, th2, fi2 = d2_sph

#S1 = tc.get_AB_arrays(np.array(r1), np.array(th1), np.array(fi1), np.array(True), 3)
SS1A, SS1B = tc.get_AB(my[:,nx], ny[:,nx], my[nx,:], ny[nx,:], np.array(r1), np.array(th1), np.array(fi1), np.array(True), J)

#S2 = tc.get_AB_arrays(np.array(r2), np.array(th2), np.array(fi2), np.array(True), 3)
SS2A, SS2B = tc.get_AB(my[:,nx], ny[:,nx], my[nx,:], ny[nx,:], np.array(r2), np.array(th2), np.array(fi2), np.array(True), J)



In [554]:
np.set_printoptions(precision=5, linewidth=200)

In [555]:
( #test for unitarity of the Wigner matrix
    np.all(np.isclose(D12_yy.conj().T,D21_yy)) and  
    np.all(np.isclose(np.dot(D12_yy,D21_yy), np.eye(nelem))) 
)

True

In [556]:
# The "vector" part
S2nA = np.dot(np.dot(D21_yy, SS1A), D12_yy) # FIXME!!!
#S2nA = np.dot(np.dot(D12_yy, SS1A), D21_yy) # FIXME!!!
np.amax(np.abs(S2nA-SS2A))

6.9388939039072284e-17

In [552]:
th2

1.0824445654829598

In [537]:
np.angle(S2nA)

array([[ -2.78373e-20,   1.49223e+00,   2.98446e+00,  -1.49223e+00,  -3.14159e+00,  -1.64936e+00,  -1.57132e-01,   1.33510e+00],
       [ -1.49223e+00,   3.66203e-19,  -1.64936e+00,  -2.98446e+00,   1.64936e+00,   6.16439e-19,   1.49223e+00,   2.98446e+00],
       [ -2.98446e+00,   1.64936e+00,  -4.74165e-20,   1.80649e+00,   1.57132e-01,  -1.49223e+00,  -3.14159e+00,  -1.64936e+00],
       [ -1.64936e+00,  -1.57132e-01,   1.33510e+00,   2.80080e-18,  -1.64936e+00,  -1.57132e-01,   1.33510e+00,   2.82733e+00],
       [  2.22086e-18,   1.49223e+00,   2.98446e+00,   1.64936e+00,  -1.04767e-18,   1.49223e+00,   2.98446e+00,  -1.80649e+00],
       [ -1.49223e+00,  -3.14159e+00,  -1.64936e+00,   1.57132e-01,  -1.49223e+00,   1.43267e-18,  -1.64936e+00,  -1.57132e-01],
       [ -2.98446e+00,   1.64936e+00,   1.13616e-18,  -1.33510e+00,  -2.98446e+00,   1.64936e+00,  -7.91427e-19,   1.49223e+00],
       [  1.80649e+00,   1.57132e-01,  -1.49223e+00,  -2.82733e+00,   1.80649e+00,   1.57132e-01,

In [538]:
np.angle(SS2A)

array([[ 0.     ,  1.49223,  2.98446, -1.49223,  3.14159, -1.64936, -0.15713,  1.3351 ],
       [-1.49223,  0.     , -1.64936, -2.98446,  1.64936,  0.     ,  1.49223,  2.98446],
       [-2.98446,  1.64936,  0.     ,  1.80649,  0.15713, -1.49223,  3.14159, -1.64936],
       [-1.64936, -0.15713,  1.3351 ,  0.     , -1.64936, -0.15713,  1.3351 ,  2.82733],
       [ 0.     ,  1.49223,  2.98446,  1.64936,  0.     ,  1.49223,  2.98446, -1.80649],
       [-1.49223,  3.14159, -1.64936,  0.15713, -1.49223,  0.     , -1.64936, -0.15713],
       [-2.98446,  1.64936,  0.     , -1.3351 , -2.98446,  1.64936,  0.     ,  1.49223],
       [ 1.80649,  0.15713, -1.49223, -2.82733,  1.80649,  0.15713, -1.49223,  0.     ]])

In [539]:
np.angle(SS1A)

array([[ 0.     ,  1.48964,  2.97929, -1.48964,  3.14159, -1.65195, -0.1623 ,  1.32734],
       [-1.48964,  0.     , -1.65195, -2.97929,  1.65195,  0.     ,  1.48964,  2.97929],
       [-2.97929,  1.65195,  0.     ,  1.81425,  0.1623 , -1.48964,  3.14159, -1.65195],
       [-1.65195, -0.1623 ,  1.32734,  0.     , -1.65195, -0.1623 ,  1.32734,  2.81698],
       [ 0.     ,  1.48964,  2.97929,  1.65195,  0.     ,  1.48964,  2.97929, -1.81425],
       [-1.48964,  3.14159, -1.65195,  0.1623 , -1.48964,  0.     , -1.65195, -0.1623 ],
       [-2.97929,  1.65195,  0.     , -1.32734, -2.97929,  1.65195,  0.     ,  1.48964],
       [ 1.81425,  0.1623 , -1.48964, -2.81698,  1.81425,  0.1623 , -1.48964,  0.     ]])

In [540]:
np.angle(D12_yy)

array([[  0.00000e+00,  -3.14159e+00,   2.44929e-16,   0.00000e+00,   0.00000e+00,   0.00000e+00,   0.00000e+00,   0.00000e+00],
       [ -1.22465e-16,   0.00000e+00,  -3.14159e+00,   0.00000e+00,   0.00000e+00,   0.00000e+00,   0.00000e+00,   0.00000e+00],
       [ -2.44929e-16,  -1.22465e-16,   0.00000e+00,   0.00000e+00,   0.00000e+00,   0.00000e+00,   0.00000e+00,   0.00000e+00],
       [  0.00000e+00,   0.00000e+00,   0.00000e+00,   0.00000e+00,  -3.14159e+00,   2.44929e-16,  -3.14159e+00,   4.89859e-16],
       [  0.00000e+00,   0.00000e+00,   0.00000e+00,  -1.22465e-16,   0.00000e+00,  -3.14159e+00,   2.44929e-16,  -3.14159e+00],
       [  0.00000e+00,   0.00000e+00,   0.00000e+00,  -2.44929e-16,  -1.22465e-16,   0.00000e+00,  -3.14159e+00,   2.44929e-16],
       [  0.00000e+00,   0.00000e+00,   0.00000e+00,  -3.67394e-16,  -2.44929e-16,  -1.22465e-16,   0.00000e+00,  -3.14159e+00],
       [  0.00000e+00,   0.00000e+00,   0.00000e+00,  -4.89859e-16,  -3.67394e-16,  -2.44929e-16,

In [541]:
np.abs(SS1A)

array([[ 0.06885,  0.01755,  0.00478,  0.00253,  0.00455,  0.00931,  0.00406,  0.00078],
       [ 0.01755,  0.00926,  0.01755,  0.00287,  0.01254,  0.01926,  0.01254,  0.00287],
       [ 0.00478,  0.01755,  0.06885,  0.00078,  0.00406,  0.00931,  0.00455,  0.00253],
       [ 0.00084,  0.00096,  0.00026,  0.01502,  0.03473,  0.01601,  0.00498,  0.00096],
       [ 0.00152,  0.00418,  0.00135,  0.03473,  0.08551,  0.03441,  0.01816,  0.00498],
       [ 0.0031 ,  0.00642,  0.0031 ,  0.01601,  0.03441,  0.04386,  0.03441,  0.01601],
       [ 0.00135,  0.00418,  0.00152,  0.00498,  0.01816,  0.03441,  0.08551,  0.03473],
       [ 0.00026,  0.00096,  0.00084,  0.00096,  0.00498,  0.01601,  0.03473,  0.01502]])

In [542]:
np.abs(S2nA-SS2A)

array([[  2.77556e-17,   1.95156e-18,   3.51154e-18,   1.89735e-19,   9.54098e-18,   1.89348e-18,   9.69740e-19,   1.20762e-18],
       [  1.95156e-18,   1.73472e-17,   1.95156e-18,   1.78811e-18,   3.87896e-18,   2.42861e-17,   1.51788e-18,   1.78811e-18],
       [  2.68217e-18,   1.95156e-18,   2.77556e-17,   1.20030e-18,   1.76495e-18,   3.63491e-18,   1.04083e-17,   5.26285e-19],
       [  9.48677e-20,   6.59494e-19,   2.79393e-19,   2.08167e-17,   3.46945e-18,   7.15245e-18,   7.88119e-18,   1.84633e-18],
       [  3.25261e-18,   1.80203e-18,   1.09633e-18,   7.96132e-18,   5.55112e-17,   7.96132e-18,   4.33681e-18,   8.77065e-18],
       [  5.26285e-19,   6.07153e-18,   5.42101e-19,   1.07287e-17,   8.67362e-18,   6.24501e-17,   4.33681e-18,   1.41196e-17],
       [  1.10068e-18,   1.78811e-18,   3.25261e-18,   6.25839e-18,   4.90654e-18,   1.45396e-17,   5.55112e-17,   3.46945e-18],
       [  3.29747e-19,   5.52837e-19,   2.36685e-19,   1.85586e-18,   6.20937e-18,   7.41074e-18,

In [543]:
SS2A

array([[  6.88511e-02+0.j     ,   1.37697e-03+0.01749j,  -4.72210e-03+0.00075j,   1.98936e-04-0.00253j,  -4.54841e-03+0.j     ,  -7.30791e-04-0.00928j,   4.00714e-03-0.00063j,   1.82564e-04+0.00076j],
       [  1.37697e-03-0.01749j,   9.25206e-03+0.j     ,  -1.37697e-03-0.01749j,  -2.83348e-03-0.00045j,  -9.84429e-04+0.0125j ,   1.92629e-02+0.j     ,   9.84429e-04+0.0125j ,  -2.83348e-03+0.00045j],
       [ -4.72210e-03-0.00075j,  -1.37697e-03+0.01749j,   6.88511e-02+0.j     ,  -1.82564e-04+0.00076j,   4.00714e-03+0.00063j,   7.30791e-04-0.00928j,  -4.54841e-03+0.j     ,  -1.98936e-04-0.00253j],
       [ -6.63120e-05-0.00084j,   9.44492e-04-0.00015j,   6.08547e-05+0.00025j,   1.50170e-02+0.j     ,  -2.72536e-03-0.03462j,   1.58024e-02-0.0025j ,   1.16133e-03+0.00484j,  -9.11356e-04+0.0003j ],
       [  1.51614e-03+0.j     ,   3.28143e-04+0.00417j,  -1.33571e-03+0.00021j,  -2.72536e-03+0.03462j,   8.55241e-02+0.j     ,   2.70078e-03+0.03431j,  -1.79351e-02+0.00284j,  -1.16133e-03-0.0048

In [544]:
S2nA

array([[  6.88511e-02 -1.91663e-21j,   1.37697e-03 +1.74901e-02j,  -4.72210e-03 +7.48162e-04j,   1.98936e-04 -2.52687e-03j,  -4.54841e-03 -2.15827e-21j,  -7.30791e-04 -9.28245e-03j,
          4.00714e-03 -6.34885e-04j,   1.82564e-04 +7.60169e-04j],
       [  1.37697e-03 -1.74901e-02j,   9.25206e-03 +3.38813e-21j,  -1.37697e-03 -1.74901e-02j,  -2.83348e-03 -4.48932e-04j,  -9.84429e-04 +1.25041e-02j,   1.92629e-02 +1.18744e-20j,
          9.84429e-04 +1.25041e-02j,  -2.83348e-03 +4.48932e-04j],
       [ -4.72210e-03 -7.48162e-04j,  -1.37697e-03 +1.74901e-02j,   6.88511e-02 -3.26468e-21j,  -1.82564e-04 +7.60169e-04j,   4.00714e-03 +6.34885e-04j,   7.30791e-04 -9.28245e-03j,
         -4.54841e-03 -9.40289e-21j,  -1.98936e-04 -2.52687e-03j],
       [ -6.63120e-05 -8.42290e-04j,   9.44492e-04 -1.49644e-04j,   6.08547e-05 +2.53390e-04j,   1.50170e-02 +4.20596e-20j,  -2.72536e-03 -3.46173e-02j,   1.58024e-02 -2.50371e-03j,
          1.16133e-03 +4.83561e-03j,  -9.11356e-04 +2.96224e-04j],
    

In [545]:
D12_yy

array([[  1.00000e+00 +0.00000e+00j,  -7.07107e-04 -8.65956e-20j,   2.50000e-07 +6.12323e-23j,   0.00000e+00 +0.00000e+00j,   0.00000e+00 +0.00000e+00j,   0.00000e+00 +0.00000e+00j,
          0.00000e+00 +0.00000e+00j,   0.00000e+00 +0.00000e+00j],
       [  7.07107e-04 -8.65956e-20j,   1.00000e+00 +0.00000e+00j,  -7.07107e-04 -8.65956e-20j,   0.00000e+00 +0.00000e+00j,   0.00000e+00 +0.00000e+00j,   0.00000e+00 +0.00000e+00j,
          0.00000e+00 +0.00000e+00j,   0.00000e+00 +0.00000e+00j],
       [  2.50000e-07 -6.12323e-23j,   7.07107e-04 -8.65956e-20j,   1.00000e+00 +0.00000e+00j,   0.00000e+00 +0.00000e+00j,   0.00000e+00 +0.00000e+00j,   0.00000e+00 +0.00000e+00j,
          0.00000e+00 +0.00000e+00j,   0.00000e+00 +0.00000e+00j],
       [  0.00000e+00 +0.00000e+00j,   0.00000e+00 +0.00000e+00j,   0.00000e+00 +0.00000e+00j,   1.00000e+00 +0.00000e+00j,  -1.00000e-03 -1.22465e-19j,   6.12372e-07 +1.49988e-22j,
         -2.50000e-10 -9.18485e-26j,   6.25000e-14 +3.06162e-29j],
    

In [546]:
1/np.sqrt(2)

0.70710678118654746

In [547]:
np.sqrt(3)

1.7320508075688772